In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from algorithms.autoencoder import autoencoder as ae
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder as onehot
from imblearn.over_sampling import RandomOverSampler

path = "/media/jahan/solo/"
os.chdir(path)

# prep data
def prep(csv):   
    labels = pd.read_csv(csv)
    labels = labels[labels["Frontal/Lateral"] == "Frontal"]
    normal = labels[labels["No Finding"] == 1]
    normal = normal.Path
    abnormal = labels[labels["No Finding"] != 1]
    abnormal = abnormal.Path
    print(len(normal),len(abnormal))

    x_a = pd.concat([normal,abnormal]).values
    indexes_switch = np.random.choice(len(x_a), int(len(x_a)/2), replace=False)
    x_b = pd.concat([normal.sample(frac=1),abnormal.sample(frac=1)]).values
    x_c = pd.concat([abnormal.sample(n=len(normal)),normal.sample(n=len(abnormal),replace=True)]).values

    temp_b = x_b[indexes_switch]
    temp_c = x_c[indexes_switch]
    x_b[indexes_switch] = temp_c
    x_c[indexes_switch] = temp_b
    
    x = np.stack((x_a,x_b,x_c), axis=-1)
    
    y = np.full((len(x_a),2),[0,1])
    y[indexes_switch] = np.full((len(indexes_switch),2),[1,0])
    
    sm = RandomOverSampler(random_state=42)
    X_res, y_res = sm.fit_resample(x, y)
    
    oh = onehot()
    y = oh.fit_transform(y_res)
    
    return X_res[:,0],X_res[:,1],X_res[:,2], y.toarray()

x_a,x_b,x_c, y = prep("CheXpert-v1.0-small/train.csv")

print(y)

config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


save_path = "/home/jahan/Documents/CXR_CNN/output/triple"


encoder = ae(a_paths=x_a, b_paths=x_b, c_paths=x_c, labels=y, save_path=save_path)
#encoder.build_dataset(batch_size=32)
encoder.build()

16974 174053


/home/jahan/miniconda3/envs/cxr/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
W0815 20:38:07.799201 140326057015104 deprecation.py:506] From /home/jahan/miniconda3/envs/cxr/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it 

[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 416         input_1[0][0]                    
                                                                 input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          conv2d[0][0]                     
                                  

In [ ]:
encoder.train(10)

W0815 20:38:17.257627 140326057015104 deprecation.py:323] From /home/jahan/miniconda3/envs/cxr/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
   847/800000 [..............................] - ETA: 128:51:31 - loss: 1.1600 - binary_crossentropy: 1.1600 - auc: 0.6986 - binary_accuracy: 0.6542

In [ ]:
x_a,x_b,x_v, y = prep("CheXpert-v1.0-small/valid.csv")

